In [ ]:
import paramiko
import os
import stat
from tqdm import tqdm

In [ ]:
hostname = 'bridges2.psc.edu'
username = 'amanov'
password = '110388+neuro'

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(hostname, username=username, password=password)

In [ ]:
directory_path = '/ocean/projects/med230011p/shared/data_rep/abcd/2yrfollowup/trks'

In [ ]:
def find_empty_files_ssh(ssh, directory):
    empty_files = []
    
    sftp = ssh.open_sftp()
    
    def check_directory(path):
        for item in tqdm(sftp.listdir_attr(path), desc="checking files in " + os.path.basename(path)):
            item_path = os.path.join(path, item.filename)
            if stat.S_ISDIR(item.st_mode):
                check_directory(item_path)
            else:
                if item.st_size == 0:
                    empty_files.append(item_path)
    
    check_directory(directory)
    sftp.close()
    
    return empty_files


In [6]:
empty_files = find_empty_files_ssh(ssh, directory_path)

if empty_files:
    print("empty files:")
    for file in empty_files:
        print(file)
else:
    print("no empty files")

ssh.close()

